# NCBIに登録されているバリアントをもとに、存在するイントロンととし得ないものの2群に分けてXGboostで特徴を抽出する
**目的**<br>
スプライスによって切り出され得るイントロンとそうではないイントロンの2群に分ける<br>
それぞれのイントロンの両端n塩基を抽出する<br>
XGboostを使用し、2群を分けるような特徴的な配列の抽出を試みる

In [ ]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
from lib.geneinfo import gene_id

# モジュールのインポート
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**変数の設定**

In [ ]:
# 解析する塩基数を指定
n = 10

# XGBoostに関する変数の設定
early_stopping_rounds = 10
learning_rate = 0.01
max_depth = 8
x_random_state = 1
s_random_state =  0
test_size = 0.2

# 解析範囲の設定
exon_start = 40
exon_end = 230

In [ ]:
gene_id('TTN')

**配列情報の取得**

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
# gbk.read_gbk('../data/gbk/human_ttn.gb')
gbk.get_gbk('7273')

# 各種バリアントの可視化
# gbk.transcript_variants()

In [ ]:
# バリアントのリストを作成 
var = gbk.get_mrna_ids()

# 特殊なバリアントを除外する
rm_list = ["XM_024453100.2","NM_133379.5", "XM_024453099.1", "NM_003319.4", "XM_017004823.1", "NM_133432.3", "NM_133437.4"]
for i in rm_list:
    var.remove(i)

**２群の作成**

In [ ]:
# すべてのエクソンの配列をvalueとする辞書を作成
# エクソン番号は、バリアントに関わらず統一的なナンバリングにする

all_exon_list = []
for i in var:
    gbk.set_mrna_id(i)
    for j in gbk.exon_list():
        if j not in all_exon_list:
            all_exon_list.append(j)

all_exon_list.sort(key=lambda x: x[0])

exon_dic = {}
for i in range(len(all_exon_list)):
    exon_dic[i+1] = all_exon_list[i]

In [ ]:
# 各バリンアントのおける統一的エクソンナンバーの組み合わせを作成
def get_key(value):
    for i in exon_dic.keys():
        if exon_dic[i] == value:
            return i

variant_dic = {}
for v in var:
    gbk.set_mrna_id(v)
    tmp_list = []
    for i in gbk.exon_list():
            tmp_list.append(get_key(i))
    variant_dic[v] = tmp_list


In [ ]:
# 存在しうるエクソンの組み合わせを作成
exon_comb = []
for i in var:
    ls = variant_dic[i]
    for i in range(len(ls)-1):
        exon_comb.append((ls[i], ls[i+1]))
exon_comb = set(exon_comb)

In [ ]:
# exon40からexon230までの範囲に解析を限定する
exon_comb_ltd = []
for i,j in exon_comb:
    if i >= exon_start and j <= exon_end:
        exon_comb_ltd.append((i,j))

existent_comb = sorted(exon_comb_ltd)
print("exon_comb_ltdの要素数:", len(existent_comb))
# exon_comb_ltd

In [ ]:
#  特殊なスプライスを受けるエクソン
for i in exon_dic.keys():
    if i < len(exon_dic.keys())-1:
        if exon_dic[i][1] > exon_dic[i+1][0]:
            print(i)

In [ ]:
# エクソンを一つ以上飛ばす組み合わせを作成
exon_distant_comb = []
for i,j in existent_comb:
    if j - i > 1:
        exon_distant_comb.append((i,j))
# exon_distant_comb

In [ ]:
# スキップされるエクソンの組み合わせを作成
space_filling_comb = []
for i,j in exon_distant_comb:
    for k in range(j-i):
        space_filling_comb.append((i,i+k+1))

space_filling_comb = set(space_filling_comb)

# 存在し得ないイントロンの組み合わせを作成
non_existent_comb = []
for i in space_filling_comb:
    if i not in existent_comb:
        non_existent_comb.append(i)
non_existent_comb = sorted(non_existent_comb)

In [ ]:
# エクソンの組み合わせがあるイントロンの始点と終点のリスト
existent_comb_region = []
for i,j in existent_comb:
    existent_comb_region.append((exon_dic[i][1], exon_dic[j][0]))
# existent_comb_region

# エクソンの組み合わせが無いイントロンの始点と終点のリスト
non_existent_comb_region = []
for i,j in non_existent_comb:
    non_existent_comb_region.append((exon_dic[i][1], exon_dic[j][0]))
# non_existent_comb_region

In [ ]:
# TTN遺伝子全長配列
seq = str(gbk.gDNA_seq())

# existent_comb_regionのイントロン両端のn塩基の配列を取得
existent_comb_region_edges = []
for i,j in existent_comb_region:
    left = seq[i:i+n]
    right = seq[j-n:j]
    existent_comb_region_edges.append((left+right))
# existent_comb_region_edges

# non_existent_comb_regionのイントロン両端のn塩基の配列を取得
non_existent_comb_region_edges = []
for i,j in non_existent_comb_region:
    left = seq[i:i+n]
    right = seq[j-n:j]
    non_existent_comb_region_edges.append((left+right))
# non_existent_comb_region_edges

**XGboostで特徴を抽出**

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
existent_comb_region_edges_arr = np.array([one_hot_encode(seq) for seq in existent_comb_region_edges])
non_existent_comb_region_edges_arr = np.array([one_hot_encode(seq) for seq in non_existent_comb_region_edges])

# 学習データとしてワンホとエンコーディングされた配列を結合しXとする
# ラベルをyとして結合する
X = pd.DataFrame(np.concatenate([existent_comb_region_edges_arr, non_existent_comb_region_edges_arr]))
y = pd.DataFrame(np.concatenate([np.ones(len(existent_comb_region_edges_arr)), np.zeros(len(non_existent_comb_region_edges_arr))]))

# XGboostのインスタンス化
model = XGBClassifier(early_stopping_rounds=early_stopping_rounds, learning_rate=learning_rate, max_depth=max_depth, random_state=x_random_state)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=s_random_state)

# データをモデルにfitさせる
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)    

**特徴の可視化**

In [ ]:
# logomakerを用いてモチーフを可視化
df = pd.DataFrame(model.feature_importances_.reshape(n*2,4))
df.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(df, shade_below=.5, fade_below=.5)

In [ ]:
# 範囲の指定
vmin, vmax = 0, 1

# ヒートマップの描画
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 2))

# 最初のヒートマップ
e_df = pd.DataFrame([list(i) for i in existent_comb_region_edges])
sns.heatmap(e_df.apply(pd.Series.value_counts).fillna(0).astype(int)/len(existent_comb_region_edges), ax=ax1, cmap="viridis", vmin=vmin, vmax=vmax, cbar=False)
ax1.set_title("existent_comb_region_edges")

# 2つ目のヒートマップ
n_df = pd.DataFrame([list(i) for i in non_existent_comb_region_edges])
cax = fig.add_axes([0.92, 0.12, 0.02, 0.76])  # カラーバーの位置とサイズを調整
sns.heatmap(n_df.apply(pd.Series.value_counts).fillna(0).astype(int)/len(non_existent_comb_region_edges), ax=ax2, cmap="viridis",vmin=vmin, vmax=vmax, cbar_ax=cax)
ax2.set_title("non_existent_comb_region_edges")

In [ ]:
# 3'側から3塩基目が"A"である割合を算出
pos = -3
base = "A"

e_count = 0
for i in existent_comb_region_edges:
    if i[pos] == base:
        e_count += 1
print("existent combination: ", round(e_count/len(existent_comb_region_edges),2))
        
n_count = 0
for i in non_existent_comb_region_edges:
    if i[pos] == base:
        n_count += 1
print("non existent combination: ", round(n_count/len(non_existent_comb_region_edges),2))
